In [1]:
import sys
from langchain.llms import Replicate
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders.csv_loader import CSVLoader
import chromadb

# Load documents from CSV file
loader = CSVLoader(file_path="./transcript/fine_food_reviews.csv")
documents = loader.load()

# Split loaded documents into smaller chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

# Create HuggingFace embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create a ChromaDB client
client = chromadb.Client()

# Create a ChromaDB collection
collection = client.create_collection(name="my_collection")

# Index the documents
print(embeddings)
for text in texts:
    embedding = embeddings(text)
    collection.add(text, embedding)

# Initialize Replicate with LLaMA3 model
llm = Replicate(
    model="a16z-infra/llama13b-v2-chat:df7690f1994d94e96ad9d568eac121aecf50684a0b0963b25a41cc40061269e5",
    input={"temperature": 0.75, "max_length": 3000}
)

# Create a QA chain using LLaMA3 and the ChromaDB collection
def retrieve(query):
    query_embedding = embeddings.encode(query)
    results = collection.query(query_embedding, k=2)
    return [(result.id, result.score) for result in results]

qa_chain = ConversationalRetrievalChain.from_llm(llm, retrieve, return_source_documents=True)

# Interact with chatbot
chat_history = []
while True:
    query = input('Prompt: ')
    if query == "exit" or query == "quit" or query == "q":
        print('Exiting')
        sys.exit()
    result = qa_chain({'question': query, 'chat_history': chat_history})
    print('Answer:', result['answer'] + '\n')
    chat_history.append((query, result['answer']))

/home/alysson/workspace/rag-with-chromadb-llama-index-ollama-csv/.venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/home/alysson/workspace/rag-with-chromadb-llama-index-ollama-csv/.venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
) model_name='sentence-transformers/all-MiniLM-L6-v2' cache_folder=None model_kwargs={} encode_kwargs={} multi_process=False show_progress=False


TypeError: 'HuggingFaceEmbeddings' object is not callable